In [32]:
import pandas as pd
import matplotlib 
import sklearn 
import sksurv
import numpy as np
import scipy.stats as stats
from scipy.stats import chisquare 

In [5]:
df = pd.read_csv("forecast_starting_data.csv")
df

,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,charge_off_reason_code,mob,open_closed_flag,...,due_balance_2,due_balance_3,due_balance_4,due_balance_5,due_balance_6,due_balance_7,due_balance_8,snapshot,mth_code,industry
0,1.0,T,0,0,I 0-6,1.0,761+,NaN,42,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,B
1,0.0,P,0,0,A,1.0,761+,NaN,48,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,B
2,0.0,C,0,0,I 0-6,0.0,761+,NaN,244,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,A
3,1.0,R,0,0,I 0-6,1.0,621-640,NaN,76,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,A
4,1.0,P,0,0,I 0-6,1.0,761+,NaN,143,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19861,1.0,I,1,0,I 13-24,1.0,621-640,NaN,26,1.0,...,1363.04,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,A
19862,0.0,R,1,0,A,1.0,641-660,NaN,299,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,C
19863,1.0,I,0,0,I 0-6,1.0,741-760,NaN,4,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,C
19864,1.0,P,1,1,A,1.0,701-720,NaN,24,1.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,202001,202001,B


In [6]:
training_data = pd.read_csv("training_data.csv")
training_data[training_data['charge_off'] == 0]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,charge_off_reason_code,mob,open_closed_flag,...,due_balance_2,due_balance_3,due_balance_4,due_balance_5,due_balance_6,due_balance_7,due_balance_8,snapshot,mth_code,industry
0,1.0,P,0,0,I 0-6,1.0,581-600,NaN,3,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201807,B
1,0.0,P,0,0,I 0-6,1.0,721-740,NaN,159,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201906,C
2,1.0,T,0,0,I 7-12,1.0,761+,NaN,22,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201905,A
3,1.0,T,0,1,I 0-6,1.0,761+,NaN,16,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201806,B
4,1.0,P,0,0,A,0.0,721-740,NaN,3,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201902,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5758218,0.0,T,1,0,A,1.0,641-660,NaN,41,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201912,201912,A
5758219,1.0,I,1,0,A,1.0,701-720,NaN,47,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201912,201912,C
5758220,1.0,R,1,0,I 0-6,1.0,741-760,NaN,34,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201912,201912,B
5758221,0.0,T,1,0,Recovery,1.0,681-700,NaN,50,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201912,201912,B


In [7]:
training_data[training_data['charge_off'] == 1]

,financial_active,net_payment_behaviour_tripd,promotion_flag,variable_rate_index,account_status_code,active_12_mths,bank_fico_buckets_20,charge_off_reason_code,mob,open_closed_flag,...,due_balance_2,due_balance_3,due_balance_4,due_balance_5,due_balance_6,due_balance_7,due_balance_8,snapshot,mth_code,industry
337775,1.0,D,1,1,A,1.0,641-660,NaN,8,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201802,C
337776,1.0,P,1,0,A,1.0,<= 560,NaN,13,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201903,A
337777,1.0,D,0,0,I 0-6,1.0,641-660,NaN,27,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201904,A
337778,0.0,D,1,0,A,1.0,621-640,NaN,2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201803,C
337779,1.0,D,0,0,A,1.0,<= 560,NaN,20,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201801,201808,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5726233,1.0,D,1,0,A,1.0,561-580,NaN,56,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201911,201912,A
5726234,1.0,D,0,0,A,1.0,581-600,NaN,230,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201911,201912,C
5726235,1.0,D,0,0,A,1.0,561-580,NaN,60,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201911,201912,B
5726236,1.0,D,1,0,A,1.0,<= 560,NaN,121,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,201911,201912,A


In [68]:
categorical = ["financial active", "net_payment_behaviour_tripd", "promotion_flag", "account_status_code", "ever_delinquent_flg", "active_12_mths",
                "open_closed_flag", "purchase_active", "closed", "active", "charge_off", "charge_off_age", "charge_off_bk", "writeoff_type_bko", 
                "writeoff_type_fraud_kiting", "writeoff_type_fraud_synthetic", "writeoff_type_deceased", "writeoff_type_other", "writeoff_type_aged",
                "writeoff_type_settlement", "writeoff_type_fraud_other", "writeoff_type_repo", "writeoff_type_null", "industry"]

categorical 

['financial active',
 'net_payment_behaviour_tripd',
 'promotion_flag',
 'account_status_code',
 'ever_delinquent_flg',
 'active_12_mths',
 'open_closed_flag',
 'purchase_active',
 'closed',
 'active',
 'charge_off',
 'charge_off_age',
 'charge_off_bk',
 'writeoff_type_bko',
 'writeoff_type_fraud_kiting',
 'writeoff_type_fraud_synthetic',
 'writeoff_type_deceased',
 'writeoff_type_other',
 'writeoff_type_aged',
 'writeoff_type_settlement',
 'writeoff_type_fraud_other',
 'writeoff_type_repo',
 'writeoff_type_null',
 'industry']

In [67]:
data_crosstab = pd.crosstab(training_data['ever_delinquent_flg'],
                            training_data['charge_off'],
                           margins=True, margins_name="Total")

data_crosstab

charge_off,0,1,Total
ever_delinquent_flg,,,
0.0,5116718,13085,5129803
1.0,625807,2613,628420
Total,5742525,15698,5758223


In [62]:
test_list = [data_crosstab, data_crosstab]
for table in test_list:
    print(table)



charge_off                 0      1    Total
ever_delinquent_flg                         
0.0                  5116718  13085  5129803
1.0                   625807   2613   628420
Total                5742525  15698  5758223
charge_off                 0      1    Total
ever_delinquent_flg                         
0.0                  5116718  13085  5129803
1.0                   625807   2613   628420
Total                5742525  15698  5758223


In [37]:
chi_square = 0
rows = training_data['ever_delinquent_flg'].unique()
columns = training_data['charge_off'].unique()
for i in columns:
    for j in rows:
        O = data_crosstab[i][j]
        E = data_crosstab[i]['Total'] * data_crosstab['Total'][j] / data_crosstab['Total']['Total']
        chi_square += (O-E)**2/E

chi_square

531.9465241903131

In [39]:
rows

array([0., 1.])

In [40]:
stats.chi2.cdf(chi_square, 1)

1.0

In [8]:
macro_data = pd.read_csv("macro_data.csv")
macro_data

,Mnemonic,M_FHOFHOPIPOQ.IUSA,M_FYLE.IUSA,M_FRT.IUSA,M_FYPSAVQ.IUSA,M_FC.IUSA,FYPCPICA_FEDB.IUSA,M_FGDP.IUSA,M_FYHHAVGQ.IUSA,M_FYFMMEDQ.IUSA,...,M_FBCFCOFCCQ.IUSA,M_FETP.IUSA,M_FLTV.IUSA,M_FC$.IUSA,M_FEIAMGTTQ.IUSA,M_FXCPMGASUQ.IUSA,M_FRBBBQ.IUSA,M_FHOWNRQ.IUSA,M_FIFR_FEDS.IUSA,M_FIFR$_FEDS.IUSA
0,Description,Baseline Scenario (October 2022): FHFA Purchas...,Baseline Scenario (October 2022): NIPA: Income...,Baseline Scenario (October 2022): Retail Sales...,Baseline Scenario (October 2022): Income: Pers...,Baseline Scenario (October 2022): NIPA: Person...,Baseline Scenario (October 2022): Income: Per ...,Baseline Scenario (October 2022): NIPA: Gross ...,Baseline Scenario (October 2022): Average Hous...,Baseline Scenario (October 2022): Median Famil...,...,Baseline Scenario (June 2020): Charge Off Rate...,Baseline Scenario (June 2020): Employment: Tot...,Baseline Scenario (June 2020): Terms Conventio...,Baseline Scenario (June 2020): NIPA: Personal ...,Baseline Scenario (June 2020): U.S. All Grades...,Baseline Scenario (June 2020): Motor gasoline ...,Baseline Scenario (June 2020): Merrill Lynch 1...,Baseline Scenario (June 2020): Homeownership R...,FRB CCAR 2020 - Severely Adverse: NIPA: Fixed ...,FRB CCAR 2020 - Severely Adverse: NIPA: Fixed ...
1,Name,"FHFA Purchase-Only Home Price Index, (Index 19...","NIPA: Income - Compensation of Employees, (Bil...","Retail Sales - Retail Sales Total, (Bil. USD, ...","Income: Personal - Saving Rate, (%, SA)",NIPA: Personal Consumption Expenditures - Tota...,"Income: Per Capita Income, (USD)","NIPA: Gross Domestic Product, (Bil. USD, SAAR)","Average Household Income: All Races, (USD, SA)","Median Family Income: All Races, (USD, SA)",...,Charge Off Rates: All Commercial Banks - Credi...,"Employment: Total Private, (Mil. #, SA)",Terms Conventional Mortgages: All Loans - Comp...,NIPA: Personal Consumption Expenditures - Tota...,U.S. All Grades All Formulations Retail Gasoli...,"Motor gasoline prices: Unleaded, (Cents per ga...","Merrill Lynch 10-Year Bbb Corporate Bond Rate,...","Homeownership Rate, (%, SA)","NIPA: Fixed Investment - Residential - Total, ...","NIPA: Fixed Investment - Residential - Total, ..."
2,Source,U.S. Federal Housing Finance Agency (FHFA); Fr...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Census Bureau (BOC); Moody's Analytics Fo...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Census Bureau (BOC); U.S. Bureau of Econo...,U.S. Census Bureau (BOC); Moody's Analytics Fo...,...,U.S. Board of Governors of the Federal Reserve...,U.S. Bureau of Labor Statistics (BLS); Moody's...,U.S. Federal Housing Finance Board (FHFB); Moo...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Energy Information Administration (EIA); ...,Oil Price Information Service (OPIS); Moody's ...,U.S. Board of Governors of the Federal Reserve...,U.S. Census Bureau (BOC); Moody's Analytics Fo...,U.S. Bureau of Economic Analysis (BEA); Moody'...,U.S. Bureau of Economic Analysis (BEA); Moody'...
3,Native Frequency,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,ANNUAL,MONTHLY,MONTHLY,MONTHLY,...,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY,MONTHLY
4,Geography,United States,United States,United States,United States,United States,United States,United States,United States,United States,...,United States,United States,United States,United States,United States,United States,United States,United States,United States,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,8/31/2035,517.343026,24477.50421,10683.55223,5.850066485,30258.36203,109631.1675,44455.57687,272055.4718,138308.4502,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435,9/30/2035,518.9847037,24562.9831,10715.48389,5.856203152,30357.61418,109972.5749,44603.62923,272841.9069,138661.205,...,NaN,NaN,NaN,NaN,NaN,NaN